# Langchain - Rerieval

### ![](../img/installation-ico.png) Install

In [ ]:
pip install langchain langchain-community faiss-cpu beautifulsoup4

### ![](../img/package-ico.png) Imports

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama
from langchain_core.documents import Document
from langchain.chains.retrieval import create_retrieval_chain

### ![](../img/parametrage-ico.png) Paramétrages


In [ ]:
loader = WebBaseLoader("https://newsroom.sfeir.com/mascotte-sfeir-pourquoi-avoir-choisi-un-axolotl-")

docs = loader.load()

model = "mistral"
# model = "mixtral"
# model = "mixtral:8x7b-instruct-v0.1-q4_1"


embeddings = OllamaEmbeddings(model=model)
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [ ]:
llm = Ollama(model=model, temperature=1)
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

### ![](../img/jouer-ico.png) Exécution simple avec document

In [ ]:
document_chain.invoke({
    "input": "Donne-moi le lien entre l'axolotl et SFEIR ?",
    "context": [Document(page_content="L'axolotl est l'animal totem de SFEIR depuis 2016. Fétiche de l'entreprise, sa bouille bienveillante séduit en interne comme en externe.")]
})

### ![](../img/jouer-ico.png) Exécution avec récupérateur

In [ ]:
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "Donne-moi le lien entre l'axolotl et SFEIR ?"})
print(response["answer"])

# The axolotl is the mascot of SFEIR since 2016...